Create a pipeline for the xml files, convert the entries to csv then move them to sqlite.

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import xmltodict as xd
import string
import re

In [2]:
# #open and convert the xml to csv
# with open('data/mochada_mpesa.xml', 'r') as file:
#     data = xd.parse(file.read())


# #extracting the messages
# messages = data['smses']['sms']

# #converting to dataframe
# df = pd.DataFrame(messages)


# #save as a csv(optional)
# df.to_csv('mochada_sms.csv', index=False)

# df.tail()

With our xml file now as a csv file, we can start data wrangling.

## Data Wrangling
Lets look at the columns of in our dataset, clean them if need be, and then use move on to Exploratory Data Analysis, before training our algorithm.

In [3]:
#loading csv file
df = pd.read_csv('mochada_sms.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   @protocol        3080 non-null   int64  
 1   @address         3080 non-null   object 
 2   @date            3080 non-null   int64  
 3   @type            3080 non-null   int64  
 4   @subject         0 non-null      float64
 5   @body            3080 non-null   object 
 6   @toa             0 non-null      float64
 7   @sc_toa          0 non-null      float64
 8   @service_center  3080 non-null   int64  
 9   @read            3080 non-null   int64  
 10  @status          3080 non-null   int64  
 11  @locked          3080 non-null   int64  
 12  @date_sent       3080 non-null   int64  
 13  @sub_id          3080 non-null   int64  
 14  @readable_date   3080 non-null   object 
 15  @contact_name    3080 non-null   object 
dtypes: float64(3), int64(9), object(4)
memory usage: 385.1+ KB


In [4]:
df.head(10)

,@protocol,@address,@date,@type,@subject,@body,@toa,@sc_toa,@service_center,@read,@status,@locked,@date_sent,@sub_id,@readable_date,@contact_name
0,0,MPESA,1713512313950,1,NaN,SDJ7N53FS3 Confirmed. On 19/4/24 at 10:38 AM T...,NaN,NaN,254722500040,1,-1,0,1713512297000,2,"Apr 19, 2024 10:38:33 AM",(Unknown)
1,0,MPESA,1713512587737,1,NaN,SDJ9N5M21L Confirmed. On 19/4/24 at 10:42 AM T...,NaN,NaN,254722500040,1,-1,0,1713512571000,2,"Apr 19, 2024 10:43:07 AM",(Unknown)
2,0,MPESA,1713512589079,1,NaN,SDJ9MU5DIL Confirmed. On 19/4/24 at 8:57 AM Ta...,NaN,NaN,254722500166,1,-1,0,1713506260000,2,"Apr 19, 2024 10:43:09 AM",(Unknown)
3,0,MPESA,1713512590679,1,NaN,SDJ2MYL302 Confirmed. On 19/4/24 at 9:39 AM Ta...,NaN,NaN,254722500040,1,-1,0,1713508798000,2,"Apr 19, 2024 10:43:10 AM",(Unknown)
4,0,MPESA,1713512593038,1,NaN,SDJ4N12JZY Confirmed. On 19/4/24 at 10:03 AM T...,NaN,NaN,254722500166,1,-1,0,1713510188000,2,"Apr 19, 2024 10:43:13 AM",(Unknown)
5,0,MPESA,1713513090993,1,NaN,SDJ4N6KYGK Confirmed. On 19/4/24 at 10:51 AM T...,NaN,NaN,254722500040,1,-1,0,1713513074000,2,"Apr 19, 2024 10:51:30 AM",(Unknown)
6,0,MPESA,1713515145278,1,NaN,SDJ3NAIRL9 confirmed.You bought Ksh50.00 of ai...,NaN,NaN,254722500041,1,-1,0,1713515117000,1,"Apr 19, 2024 11:25:45 AM",(Unknown)
7,0,MPESA,1713517198429,1,NaN,SDJ4NEIHH0 Confirmed. On 19/4/24 at 11:59 AM T...,NaN,NaN,254722500040,1,-1,0,1713517182000,2,"Apr 19, 2024 11:59:58 AM",(Unknown)
8,0,MPESA,1713517454715,1,NaN,SDJ6NF0IHS Confirmed. On 19/4/24 at 12:03 PM T...,NaN,NaN,254722500040,1,-1,0,1713517438000,2,"Apr 19, 2024 12:04:14 PM",(Unknown)
9,0,MPESA,1713518092454,1,NaN,SDJ2NG9PAO Confirmed. On 19/4/24 at 12:14 PM T...,NaN,NaN,254722500040,1,-1,0,1713518076000,2,"Apr 19, 2024 12:14:52 PM",(Unknown)


Based on the output from the last cell, here is a general outline of what we will do:
- Create a new dataframe with only three main columns; Address, Body, and readable date
- Change readable date to pandas datetime object
- Inspect and see what information to extract from the body ie balance amount, amount taken or given
- Perform a weekly analysis from Wednesday to Tuesday
- Calculate the monthly commission earned based on the transactions
- Do a time series analysis of the dataset

In [5]:
#columns to keep
keep = ['@address', '@body', '@readable_date']

#new data frame 
data = df[keep].copy()

#renaming the columns 
data.columns = ['address', 'body', 'date']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  3080 non-null   object
 1   body     3080 non-null   object
 2   date     3080 non-null   object
dtypes: object(3)
memory usage: 72.3+ KB


In [6]:
#converting the date into datetime object
data['date'] = pd.to_datetime(data['date'], format= 'mixed')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   address  3080 non-null   object        
 1   body     3080 non-null   object        
 2   date     3080 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 72.3+ KB


In [7]:
data.tail()

,address,body,date
3075,MPESA,SGS245L5GQ Confirmed. On 28/7/24 at 10:55 AM T...,2024-07-28 10:55:57
3076,MPESA,SGS8488HBK Confirmed. on 28/7/24 at 11:20 AM G...,2024-07-28 11:20:05
3077,MPESA,SGS449HBC6 Confirmed. On 28/7/24 at 11:31 AM T...,2024-07-28 11:31:31
3078,MPESA,SGS949ZVIH Confirmed. On 28/7/24 at 11:35 AM T...,2024-07-28 11:36:22
3079,MPESA,SGS04DA44U Confirmed. On 28/7/24 at 12:05 PM T...,2024-07-28 12:05:47


With the new dataframe created, it seems most of the information is contained in the `body` column. Lets load one entry of this column and see what information is relevant to be extracted.

In [28]:
data['body'].iloc[6]

'SDJ3NAIRL9 confirmed.You bought Ksh50.00 of airtime on 19/4/24 at 11:25 AM.New M-PESA balance is Ksh738.04. Transaction cost, Ksh0.00. Amount you can transact within the day is 499,950.00. Reverse erroneously purchased airtime using M-PESA by sending the M-PESA SMS to 456'

There is a unique `transaction ID` for each entry, then the status; `confirmed` or `failed`, action; `take` or `give`, the `amount`, `name` of the customer and the `balance` remaining. 

Lets write a function to help us extract them and use them for analysis.

In [29]:
def cleaner(x):
    parts = x.split()
    final = []
    amount_pattern = r'Ksh[\d,]+\.\d{2}'
    for part in parts:
        if re.match(amount_pattern, part):
            final.append(part)
        else:
            part = part.replace('.', ' ')
            subsplits = part.split()
            subsplits = [word.strip(string.punctuation).upper() for word in part.split()]
            final.extend(subsplits)
    return final

trying = data['body'].apply(cleaner).apply(pd.Series)
type(trying)
trying.head()

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,SDJ7N53FS3,CONFIRMED,ON,19/4/24,AT,10:38,AM,TAKE,"Ksh1,100.00",CASH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SDJ9N5M21L,CONFIRMED,ON,19/4/24,AT,10:42,AM,TAKE,"Ksh1,100.00",CASH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SDJ9MU5DIL,CONFIRMED,ON,19/4/24,AT,8:57,AM,TAKE,"Ksh1,100.00",CASH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SDJ2MYL302,CONFIRMED,ON,19/4/24,AT,9:39,AM,TAKE,Ksh100.00,CASH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SDJ4N12JZY,CONFIRMED,ON,19/4/24,AT,10:03,AM,TAKE,Ksh300.00,CASH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Phase one of cleaning is done, now lets move to phase 2. Where we will drop unnecessary columns, stnadardize the needed ones by making sure the data types are as they should be.

In [41]:
count = 0

for _ in trying[0]:
    if _ == 'M-PESA':
        count += 1

count

7

In [ ]:
""" 
This function takes a row as input, 
goes through each of the entries and populates the money column with 
"""

def cleaner2(x):
    ...

In [11]:
data['body'].iloc[10]

'SDJ6NL185I Confirmed. On 19/4/24 at 12:54 PM Take Ksh5,500.00 cash from MWENDWA KANYAI Your M-PESA float balance is Ksh71,087.00. Click the link to Download M-Pesa Agent App and Transact the SMART way https://bit.ly/3Ll6JQU'

In [12]:
""" 
There seems to be inconsistencies in the entries, thus a more robust function or series of functions is 
needed.
Brainstorming sessions:
    1. look for patterns for the similar entries and filter based on that
    2. Second option is: after splitting it into the different sections, go over each of the entries and split them 
    again on every punctuation mark.
"""

' \nThere seems to be inconsistencies in the entries, thus a more robust function or series of functions is \nneeded.\nBrainstorming sessions:\n    1. look for patterns for the similar entries and filter based on that\n    2. Second option is: after splitting it into the different sections, go over each of the entries and split them \n    again on every punctuation mark.\n'

In [13]:
# import pandas as pd
# import string

# class StringProcessor:
    
#     def spli(self, x):
#         parts = x.split()
#         part_1 = parts[:5]
#         part_2 = parts[5:11]
#         part_3 = parts[11:16]
#         part_4 = parts[16:20]
#         return part_1, part_2, part_3, part_4

#     def split_and_capitalize(self, parts):
#         processed_parts = []
#         for part in parts:
#             # Split each sublist by punctuation marks
#             split_subparts = [word.strip(string.punctuation).capitalize() for sublist in part for word in sublist.split()]
#             processed_parts.append(split_subparts)
#         return processed_parts

#     def to_dataframe(self, processed_parts):
#         # Create a dictionary with keys as column names and values as the lists of processed parts
#         data_dict = {f'Part_{i+1}': pd.Series(part) for i, part in enumerate(processed_parts)}
#         df = pd.DataFrame(data_dict)
#         return df

# # Example usage
# processor = StringProcessor()

# # Input string
# # input_string = "This is an example string that we will use to test the functionality of the class and its methods correctly"

# # Method 1: Split input string
# split_parts = processor.spli(sample)
# print("Split Parts:", split_parts)

# # Method 2: Split on punctuation and capitalize
# capitalized_parts = processor.split_and_capitalize(split_parts)
# print("Capitalized Parts:", capitalized_parts)

# # Method 3: Convert to DataFrame
# df = processor.to_dataframe(capitalized_parts)
# print("DataFrame:\n", df)


In [14]:
#lets try option 2 before going back to option one
import string

class StringProcessor:
    def spli(self, x):
        parts = x.split()
        part_1 = parts[:5]
        part_2 = parts[5:11]
        part_3 = parts[11:16]
        part_4 = parts[16:20]
        return part_1, part_2, part_3, part_4
    
    def split_processing(self, parts):
        processed_parts = []
        for part in parts:
            #split on punctuation marks
            split_subparts = [word.strip(string.punctuation).upper() for sublist in part for word in sublist.split()]
            processed_parts.append(split_subparts)
        return processed_parts
    
    def to_dataframe(self, processed_parts):
        ...

#initialize the class instance
processor = StringProcessor()

sample = data['body'].iloc[6]
# Method 1: Split input string
split_parts = processor.spli(sample)
# print("Split Parts:", split_parts)

# Method 2: Split on punctuation and capitalize
capitalized_parts = processor.split_processing(split_parts)
print("Capitalized Parts:", capitalized_parts)

Capitalized Parts: [['SDJ7N53FS3', 'CONFIRMED', 'ON', '19/4/24', 'AT'], ['10:38', 'AM', 'TAKE', 'KSH1,100.00', 'CASH', 'FROM'], ['ROBERT', 'MAHINDA', 'YOUR', 'M-PESA', 'FLOAT'], ['BALANCE', 'IS', 'KSH82,007.00', 'CLICK']]


In [15]:
#lets create a function to extract information from this column
""" 
Function takes the a sting as input
Strips it into different sections
Extracts specific information from the texts;
    - unique id
    - transaction status
    - transaction type
    - amount
    - customer names
    - balance
"""

import re

def extract(text):
    # Split the text
    parts = text.split()
    
    # Initialize variables
    id = parts[0]
    status = parts[1]

    # Determine the action
    if 'Take' in parts:  # Someone gave money to deposit
        action = 'Take'
        action_index = parts.index('Take')

    elif 'Give' in parts:  # Someone withdrew money
        action = 'Give'
        action_index = parts.index('Give')

    elif 'bought' in parts:  # Sold airtime to customer
        action = 'bought'
        action_index = parts.index('bought')

    elif 'sent' in parts:  # Personal transactions
        action = 'sent'
        action_index = parts.index('sent')

    else:
        action = None
        action_index = -1

    # Extraction of the amount 
    amount_pattern = r'Ksh[\d,]+\.\d{2}'
    amount_match = re.search(amount_pattern, text)
    amount = amount_match.group(0) if amount_match else None

    # Remove 'Ksh' from the amount
    if amount:
        amount = re.sub(r'Ksh\s*', '', amount)
        amount = re.sub(r',', '', amount)
        amount = float(amount)

    # Extraction of customer name
    if action in ['Take', 'Give']:
        name = parts[action_index + 4] + ' ' + parts[action_index + 5]
        name = name.upper()
    else:
        name = None

    # Extraction of Mpesa balance
    balance_start = text.find('balance is') + len('balance is') + 1
    balance_end = text.find('.', balance_start) + 3
    
    if balance_start != -1 and balance_end != -1:
        balance = text[balance_start:balance_end].strip()

        # Remove 'Kshs' from the balance
        balance = re.sub(r'Ksh\s*', '', balance)
        balance = re.sub(r',', '', balance)
        # balance = float(balance)
    else:
        balance = None

    return {
        'Transaction Id': id,
        'Status': status,
        'Action': action,
        'Amount': amount,
        'Customer Name': name,
        'balance': balance,
    }


In [16]:
new_df = data['body'].apply(extract).apply(pd.Series)

new_df.head()

,Transaction Id,Status,Action,Amount,Customer Name,balance
0,SDJ7N53FS3,Confirmed.,Take,1100.0,ROBERT MAHINDA,82007.00
1,SDJ9N5M21L,Confirmed.,Take,1100.0,BRIAN CHACHA,80907.00
2,SDJ9MU5DIL,Confirmed.,Take,1100.0,CHRISTOPHER RWARA,97007.00
3,SDJ2MYL302,Confirmed.,Take,100.0,JOHN TIMPANKO,83407.00
4,SDJ4N12JZY,Confirmed.,Take,300.0,NICKSON MIDUNGA,83107.00


In [17]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3080 entries, 0 to 3079
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Transaction Id  3080 non-null   object 
 1   Status          3080 non-null   object 
 2   Action          2725 non-null   object 
 3   Amount          2891 non-null   float64
 4   Customer Name   2491 non-null   object 
 5   balance         3080 non-null   object 
dtypes: float64(1), object(5)
memory usage: 144.5+ KB


In [18]:
def filter_and_extract_ids(df):
    # Define the valid actions
    valid_actions = ['Take', 'Give', 'bought']

    # Filter the DataFrame to only include rows with valid actions
    filtered_df = df[df['Action'].isin(valid_actions)]

    # Extract the row indices of the filtered rows
    unique_ids = filtered_df['Transaction Id'].unique().tolist()


    return filtered_df, unique_ids




In [19]:
filtered_df, unique_ids = filter_and_extract_ids(new_df)

In [20]:
filtered_df.head()

,Transaction Id,Status,Action,Amount,Customer Name,balance
0,SDJ7N53FS3,Confirmed.,Take,1100.0,ROBERT MAHINDA,82007.00
1,SDJ9N5M21L,Confirmed.,Take,1100.0,BRIAN CHACHA,80907.00
2,SDJ9MU5DIL,Confirmed.,Take,1100.0,CHRISTOPHER RWARA,97007.00
3,SDJ2MYL302,Confirmed.,Take,100.0,JOHN TIMPANKO,83407.00
4,SDJ4N12JZY,Confirmed.,Take,300.0,NICKSON MIDUNGA,83107.00


In [21]:
len(unique_ids)
# joined_df.info()


2422

In [22]:
#continue cleaning of this data. 

In [23]:
data['body'].iloc[-17]

'The customer is not registered to M-PESA and advise the agent to register the customer first.'

In [24]:
["""
 - Failed. Kindly capture the correct mobile number and customer details as they appear on the identification document and attempt the 
    deposit again.
 - SDJ9N5M21L Confirmed. On 19/4/24 at 10:42 AM Take Ksh1,100.00 cash from brian chacha Your M-PESA float balance is Ksh80,907.00. 
    Click the link to Download M-Pesa Agent App and Transact the SMART way https://bit.ly/3Ll6JQU
 - The customer is not registered to M-PESA and advise the agent to register the customer first.

 - SDJ6NL185I Confirmed. On 19/4/24 at 12:54 PM Take Ksh5,500.00 cash from MWENDWA KANYAI Your M-PESA float balance is Ksh71,087.00. 
    Click the link to Download M-Pesa Agent App and Transact the SMART way https://bit.ly/3Ll6JQU
 - SGQ7V4HPMX Confirmed. On 26/7/24 at 9:54 AM Take Ksh328.00 cash from ROSE KAKINDU Your M-PESA float balance is Ksh132,626.00. 
    Click the link to Download M-Pesa Agent App and Transact the SMART way https://bit.ly/3Ll6JQU
 - SF98YJ0DIY Confirmed. On 9/6/24 at 7:59 PM Take Ksh1,651.00 cash from CAROLINE ABUGA Your M-PESA float balance is Ksh63,721.00. 
    Click the link to Download M-Pesa Agent App and Transact the SMART way https://bit.ly/3Ll6JQU
 """]

sample = data['body'].iloc[-1520]


In [46]:
data['date'].iloc[-1520]

Timestamp('2024-06-09 19:59:43')